#**Kernel Ridge Regression Playground**

#Preprocess

In [19]:
!pip install ipywidgets --quiet

In [2]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [211]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.kernel_ridge import KernelRidge
import matplotlib as mpl
%matplotlib inline

In [7]:
from ipywidgets import interact,interactive,fixed,interact_manual
import ipywidgets as widgets

In [58]:
plt.rcParams['figure.figsize']=(6,4)
plt.rcParams['figure.dpi']=150

In [181]:
np.random.seed(1234)
X = np.random.uniform(0,10,30)
Y = np.random.uniform(-1.0,1.0,30)

#Kernels
Gaussian: $e^{-(x-y)^2}$ \\
Cosine: $cos(x-y)$ \\
Exp: $e^{x+y}$

In [204]:
def Gaussian(x,y):
  return np.exp(-(x-y)**2)
def Cosine(x,y):
  return np.cos(x-y)
def Exp(x,y):
  return np.exp(x+y)

#Playground

In [214]:
Kernels = ["Gaussian","Cos","Exp"]
@interact(kernel=Kernels,alpha=(0.0,10,0.08))
def engine(kernel,alpha):
  K = None
  s,t = np.meshgrid(X,X,indexing='ij')

  if kernel=="Gaussian":
    K = Gaussian(s,t)
  elif kernel=="Exp":
    K = Exp(s,t)
  else:
    K = Cosine(s,t)

  n = X.shape[0]

  A = np.linalg.inv(K+(n*alpha/100)*np.eye(n))
  weights = np.matmul(A,Y)
  x = np.linspace(0,10,1000)

  y = None


  if kernel=="Gaussian":
    y = np.array([w*Gaussian(i,j) for i in x for (j,w) in zip(X,weights) ])
    y=y.reshape(-1,n)
    y = np.sum(y,axis=1)

  elif kernel=="Cos":
    y = np.array([w*Cosine(i,j) for i in x for (j,w) in zip(X,weights) ])
    y=y.reshape(-1,n)
    y = np.sum(y,axis=1)

  else:
    y = np.array([w*Exp(i,j) for i in x for (j,w) in zip(X,weights) ])
    y=y.reshape(-1,n)
    y = np.sum(y,axis=1)





  fig,ax=plt.subplots(1,2,figsize=(12,4))

  if kernel=="Gaussian":
    model = KernelRidge(alpha=alpha,kernel="rbf")
    model.fit(X.reshape(-1,1),Y)
    y_pred= model.predict(x.reshape(-1,1))
    ax[0].plot(x,y_pred,color="red",label="sklearn")

  ax[0].scatter(X,Y)
  ax[0].plot(x,y,linestyle="--",color='green',label="Our Curve")

  ax[1].bar(np.arange(30),weights)
  ax[0].set(title="Data")
  ax[1].set(title="Weights")
  ax[0].legend()
  ax[0].grid()
  plt.subplots_adjust(wspace=0.5)




interactive(children=(Dropdown(description='kernel', options=('Gaussian', 'Cos', 'Exp'), value='Gaussian'), Fl…